In [1]:
import time
from datetime import date

startTime = time.time()
todaysDate = date.today()

# Run Date: Wednesday, July 05, 2023
# Run Time: 00:03:35

Monday, July 3, 2023

This notebook will contain everything needed to run nanoGPT in easily digestable chunks.

The workflow is to start with prepare.py, then train with train.py, then sample with sample.py

#### Prepare.py

In [2]:
"""
Prepare the Shakespeare dataset for character-level language modeling.
So instead of encoding with GPT-2 BPE tokens, we just map characters to ints.
Will save train.bin, val.bin containing the ids, and meta.pkl containing the
encoder and decoder and some other related info.
"""
import os
import time
import pickle
import requests
import numpy as np


In [3]:
# download the tiny shakespeare dataset
# input_file_path = os.path.join(os.path.dirname(__file__), 'input.txt')
input_file_path = 'input.txt'


In [4]:
if not os.path.exists(input_file_path):
    data_url = 'https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt'
    with open(input_file_path, 'w') as f:
        f.write(requests.get(data_url).text)

In [5]:
with open(input_file_path, 'r') as f:
    data = f.read()
print(f"length of dataset in characters: {len(data):,}")

length of dataset in characters: 1,115,394


In [6]:
# get all the unique characters that occur in this text
chars = sorted(list(set(data)))
vocab_size = len(chars)
print("all the unique characters:", ''.join(chars))
print(f"vocab size: {vocab_size:,}")


all the unique characters: 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
vocab size: 65


In [7]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }

In [8]:
def encode(s):
    return [stoi[c] for c in s] # encoder: take a string, output a list of integers

In [9]:
def decode(l):
    return ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string


In [10]:
# create the train and test splits
n = len(data)
train_data = data[:int(n*0.9)]
val_data = data[int(n*0.9):]

In [11]:
# encode both to integers
train_ids = encode(train_data)
val_ids = encode(val_data)
print(f"train has {len(train_ids):,} tokens")
print(f"val has {len(val_ids):,} tokens")

train has 1,003,854 tokens
val has 111,540 tokens


In [12]:
# export to bin files
train_ids = np.array(train_ids, dtype=np.uint16)
val_ids = np.array(val_ids, dtype=np.uint16)
# train_ids.tofile(os.path.join(os.path.dirname(__file__), 'train.bin'))
# val_ids.tofile(os.path.join(os.path.dirname(__file__), 'val.bin'))
train_ids.tofile('train.bin')
val_ids.tofile('val.bin')

In [13]:
# save the meta information as well, to help us encode/decode later
meta = {
    'vocab_size': vocab_size,
    'itos': itos,
    'stoi': stoi,
}
# with open(os.path.join(os.path.dirname(__file__), 'meta.pkl'), 'wb') as f:
#     pickle.dump(meta, f)
with open('meta.pkl', 'wb') as f:
    pickle.dump(meta, f)

#### Train.py

In [14]:
import math
import pickle
from contextlib import nullcontext

In [15]:
import torch
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.distributed import init_process_group, destroy_process_group

In [16]:
# -----------------------------------------------------------------------------
# default config values designed to train a gpt2 (124M) on OpenWebText
# I/O
out_dir = 'out'
eval_interval = 2000
log_interval = 1
eval_iters = 200
eval_only = False # if True, script exits right after the first eval
always_save_checkpoint = True # if True, always save a checkpoint after each eval
init_from = 'scratch' # 'scratch' or 'resume' or 'gpt2*'

# wandb logging
wandb_log = False # disabled by default
wandb_project = 'owt'
wandb_run_name = 'gpt2' # 'run' + str(time.time())

# data
dataset = 'openwebtext'
gradient_accumulation_steps = 5 * 8 # used to simulate larger batch sizes
batch_size = 12 # if gradient_accumulation_steps > 1, this is the micro-batch size
block_size = 1024

# model
n_layer = 12
n_head = 12
n_embd = 768
dropout = 0.0 # for pretraining 0 is good, for finetuning try 0.1+
bias = False # do we use bias inside LayerNorm and Linear layers?

# adamw optimizer
learning_rate = 6e-4 # max learning rate
max_iters = 600000 # total number of training iterations
weight_decay = 1e-1
beta1 = 0.9
beta2 = 0.95
grad_clip = 1.0 # clip gradients at this value, or disable if == 0.0

# learning rate decay settings
decay_lr = True # whether to decay the learning rate
warmup_iters = 2000 # how many steps to warm up for
lr_decay_iters = 600000 # should be ~= max_iters per Chinchilla
min_lr = 6e-5 # minimum learning rate, should be ~= learning_rate/10 per Chinchilla

# DDP settings
backend = 'nccl' # 'nccl', 'gloo', etc.

# system
device = 'cuda' # examples: 'cpu', 'cuda', 'cuda:0', 'cuda:1' etc., or try 'mps' on macbooks

# This was bfloat16, but it was blowing up ... so I set it to float16 and it works.
# dtype = 'float16' # 'float32', 'bfloat16', or 'float16', the latter will auto implement a GradScaler
dtype = 'bfloat16' if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else 'float16' # 'float32', 'bfloat16', or 'float16', the latter will auto implement a GradScaler

compile = True # use PyTorch 2.0 to compile the model to be faster

In [17]:
config_keys = [k for k,v in globals().items() if not k.startswith('_') and isinstance(v, (int, float, bool, str))]
exec(open('config/train_shakespeare_char.py').read()) # overrides from command line or config file
config = {k: globals()[k] for k in config_keys} # will be useful for logging

In [18]:
# Any manual overrides of the config file, place them here ...
wandb_log = True

In [19]:
# if not ddp, we are running on a single gpu, and one process
ddp = False
master_process = True
seed_offset = 0
ddp_world_size = 1

In [20]:
tokens_per_iter = gradient_accumulation_steps * ddp_world_size * batch_size * block_size
print(f"tokens per iteration will be: {tokens_per_iter:,}")

tokens per iteration will be: 4,096


In [21]:
if master_process:
    os.makedirs(out_dir, exist_ok=True)

In [22]:
torch.manual_seed(1337 + seed_offset)
torch.backends.cuda.matmul.allow_tf32 = True # allow tf32 on matmul
torch.backends.cudnn.allow_tf32 = True # allow tf32 on cudnn
device_type = 'cuda' if 'cuda' in device else 'cpu' # for later use in torch.autocast
# note: float16 data type will automatically use a GradScaler
ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]
ctx = nullcontext() if device_type == 'cpu' else torch.amp.autocast(device_type=device_type, dtype=ptdtype)

In [23]:
dataset

'shakespeare_char'

In [24]:
# poor man's data loader
# data_dir = os.path.join('data', dataset)
# osCwd = os.getcwd()
# data_dir = os.path.join('data', dataset)
# train_data = np.memmap(os.path.join(data_dir, 'train.bin'), dtype=np.uint16, mode='r')
# val_data = np.memmap(os.path.join(data_dir, 'val.bin'), dtype=np.uint16, mode='r')

train_data = np.memmap('train.bin', dtype=np.uint16, mode='r')
val_data = np.memmap('val.bin', dtype=np.uint16, mode='r')

In [25]:
def get_batch(split):
    
    data = train_data if split == 'train' else val_data
    
    ix = torch.randint(len(data) - block_size, (batch_size,))

    x = torch.stack([torch.from_numpy((data[i:i+block_size]).astype(np.int64)) for i in ix])
    y = torch.stack([torch.from_numpy((data[i+1:i+1+block_size]).astype(np.int64)) for i in ix])

    if device_type == 'cuda':
        # pin arrays x,y, which allows us to move them to GPU asynchronously (non_blocking=True)
        x, y = x.pin_memory().to(device, non_blocking=True), y.pin_memory().to(device, non_blocking=True)
    else:
        x, y = x.to(device), y.to(device)
    return x, y

In [26]:
# init these up here, can override if init_from='resume' (i.e. from a checkpoint)
iter_num = 0
best_val_loss = 1e9

In [27]:
# attempt to derive vocab_size from the dataset
# meta_path = os.path.join(data_dir, 'meta.pkl')
meta_path = 'meta.pkl'
meta_vocab_size = None
if os.path.exists(meta_path):
    with open(meta_path, 'rb') as f:
        meta = pickle.load(f)
    meta_vocab_size = meta['vocab_size']
    print(f"found vocab_size = {meta_vocab_size} (inside {meta_path})")

found vocab_size = 65 (inside meta.pkl)


In [28]:
# model init
model_args = dict(n_layer=n_layer, n_head=n_head, n_embd=n_embd, block_size=block_size,
                  bias=bias, vocab_size=None, dropout=dropout) # start with model_args from command line

In [29]:
from model import GPTConfig, GPT

In [30]:
if init_from == 'scratch':
    # init a new model from scratch
    print("Initializing a new model from scratch")

    # determine the vocab size we'll use for from-scratch training
    if meta_vocab_size is None:
        print("defaulting to vocab_size of GPT-2 to 50304 (50257 rounded up for efficiency)")

    model_args['vocab_size'] = meta_vocab_size if meta_vocab_size is not None else 50304
    
    gptconf = GPTConfig(**model_args)
    model = GPT(gptconf)
elif init_from == 'resume':
    print(f"Resuming training from {out_dir}")
    # resume training from a checkpoint.
    ckpt_path = os.path.join(out_dir, 'ckpt.pt')
    checkpoint = torch.load(ckpt_path, map_location=device)
    checkpoint_model_args = checkpoint['model_args']
    # force these config attributes to be equal otherwise we can't even resume training
    # the rest of the attributes (e.g. dropout) can stay as desired from command line
    for k in ['n_layer', 'n_head', 'n_embd', 'block_size', 'bias', 'vocab_size']:
        model_args[k] = checkpoint_model_args[k]
    # create the model
    gptconf = GPTConfig(**model_args)
    model = GPT(gptconf)
    state_dict = checkpoint['model']
    # fix the keys of the state dictionary :(
    # honestly no idea how checkpoints sometimes get this prefix, have to debug more
    unwanted_prefix = '_orig_mod.'
    for k,v in list(state_dict.items()):
        if k.startswith(unwanted_prefix):
            state_dict[k[len(unwanted_prefix):]] = state_dict.pop(k)
    model.load_state_dict(state_dict)
    iter_num = checkpoint['iter_num']
    best_val_loss = checkpoint['best_val_loss']
elif init_from.startswith('gpt2'):
    print(f"Initializing from OpenAI GPT-2 weights: {init_from}")
    # initialize from OpenAI GPT-2 weights
    override_args = dict(dropout=dropout)
    model = GPT.from_pretrained(init_from, override_args)
    # read off the created config params, so we can store them into checkpoint correctly
    for k in ['n_layer', 'n_head', 'n_embd', 'block_size', 'bias', 'vocab_size']:
        model_args[k] = getattr(model.config, k)

Initializing a new model from scratch
number of parameters: 10.65M


In [31]:
print(f'block_size -> {block_size}')
print(f'model.config.block_size -> {model.config.block_size}')

block_size -> 256
model.config.block_size -> 256


In [32]:
# crop down the model block size if desired, using model surgery
if block_size < model.config.block_size:
    model.crop_block_size(block_size)
    model_args['block_size'] = block_size # so that the checkpoint will have the right value

In [33]:
model.to(device)

GPT(
  (transformer): ModuleDict(
    (wte): Embedding(65, 384)
    (wpe): Embedding(256, 384)
    (drop): Dropout(p=0.2, inplace=False)
    (h): ModuleList(
      (0-5): 6 x Block(
        (ln_1): LayerNorm()
        (attn): CausalSelfAttention(
          (c_attn): Linear(in_features=384, out_features=1152, bias=False)
          (c_proj): Linear(in_features=384, out_features=384, bias=False)
          (attn_dropout): Dropout(p=0.2, inplace=False)
          (resid_dropout): Dropout(p=0.2, inplace=False)
        )
        (ln_2): LayerNorm()
        (mlp): MLP(
          (c_fc): Linear(in_features=384, out_features=1536, bias=False)
          (c_proj): Linear(in_features=1536, out_features=384, bias=False)
          (dropout): Dropout(p=0.2, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm()
  )
  (lm_head): Linear(in_features=384, out_features=65, bias=False)
)

In [34]:
# initialize a GradScaler. If enabled=False scaler is a no-op
scaler = torch.cuda.amp.GradScaler(enabled=(dtype == 'float16'))

In [35]:
# optimizer
optimizer = model.configure_optimizers(weight_decay, learning_rate, (beta1, beta2), device_type)

print(f'init_from => {init_from}')
if init_from == 'resume':
    optimizer.load_state_dict(checkpoint['optimizer'])

checkpoint = None # free up memory


using fused AdamW: True
init_from => scratch


In [36]:
%%time
# compile the model
if compile:
    print("compiling the model... (takes a ~minute)")
    unoptimized_model = model
    model = torch.compile(model) # requires PyTorch 2.0

compiling the model... (takes a ~minute)
CPU times: user 149 ms, sys: 8.26 ms, total: 157 ms
Wall time: 156 ms


In [37]:
# helps estimate an arbitrarily accurate loss over either split using many batches
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            with ctx:
                logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [38]:
# learning rate decay scheduler (cosine with warmup)
def get_lr(it):
    # 1) linear warmup for warmup_iters steps
    if it < warmup_iters:
        return learning_rate * it / warmup_iters
    # 2) if it > lr_decay_iters, return min learning rate
    if it > lr_decay_iters:
        return min_lr
    # 3) in between, use cosine decay down to min learning rate
    decay_ratio = (it - warmup_iters) / (lr_decay_iters - warmup_iters)
    assert 0 <= decay_ratio <= 1
    coeff = 0.5 * (1.0 + math.cos(math.pi * decay_ratio)) # coeff ranges 0..1
    return min_lr + coeff * (learning_rate - min_lr)


In [39]:
if wandb_log and master_process:
    os.environ['WANDB_NOTEBOOK_NAME'] = 'nanoGPT.ipynb'
    import wandb

    wandb.login(relogin=True)

    # wandb.login()

    

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [40]:
wandBconfig=config.copy()
wandBconfig['data'] = ''
wandBconfig['train_data'] = ''
wandBconfig['val_data'] = ''
wandBconfig

{'startTime': 1688588177.4510758,
 'input_file_path': 'input.txt',
 'data': '',
 'vocab_size': 65,
 'n': 1115394,
 'train_data': '',
 'val_data': '',
 'out_dir': 'out-shakespeare-char',
 'eval_interval': 250,
 'log_interval': 10,
 'eval_iters': 200,
 'eval_only': False,
 'always_save_checkpoint': False,
 'init_from': 'scratch',
 'wandb_log': False,
 'wandb_project': 'shakespeare-char',
 'wandb_run_name': 'mini-gpt',
 'dataset': 'shakespeare_char',
 'gradient_accumulation_steps': 1,
 'batch_size': 16,
 'block_size': 256,
 'n_layer': 6,
 'n_head': 6,
 'n_embd': 384,
 'dropout': 0.2,
 'bias': False,
 'learning_rate': 0.001,
 'max_iters': 5000,
 'weight_decay': 0.1,
 'beta1': 0.9,
 'beta2': 0.99,
 'grad_clip': 1.0,
 'decay_lr': True,
 'warmup_iters': 100,
 'lr_decay_iters': 5000,
 'min_lr': 0.0001,
 'backend': 'nccl',
 'device': 'cuda',
 'dtype': 'float16',
 'compile': True}

In [41]:
if wandb_log and master_process:
    import wandb
    wandb.init(project=wandb_project, name=wandb_run_name, config=wandBconfig, settings=wandb.Settings(start_method="fork"))

wandb: Currently logged in as: robkayinto. Use `wandb login --relogin` to force relogin


In [42]:
%%time

# training loop
X, Y = get_batch('train') # fetch the very first batch
t0 = time.time()
local_iter_num = 0 # number of iterations in the lifetime of this process
raw_model = model.module if ddp else model # unwrap DDP container if needed
running_mfu = -1.0

while True:

    # determine and set the learning rate for this iteration
    lr = get_lr(iter_num) if decay_lr else learning_rate
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

    # evaluate the loss on train/val sets and write checkpoints
    if iter_num % eval_interval == 0 and master_process:
        losses = estimate_loss()
        print(f"step {iter_num}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

        if wandb_log:
            wandb.log({
                "iter": iter_num,
                "train/loss": losses['train'],
                "val/loss": losses['val'],
                "lr": lr,
                "mfu": running_mfu*100, # convert to percentage
            })

        if losses['val'] < best_val_loss or always_save_checkpoint:
            best_val_loss = losses['val']
            if iter_num > 0:
                checkpoint = {
                    'model': raw_model.state_dict(),
                    'optimizer': optimizer.state_dict(),
                    'model_args': model_args,
                    'iter_num': iter_num,
                    'best_val_loss': best_val_loss,
                    'config': config,
                }
                print(f"saving checkpoint to {out_dir}")
                torch.save(checkpoint, os.path.join(out_dir, 'ckpt.pt'))

    if iter_num == 0 and eval_only:
        break

    # forward backward update, with optional gradient accumulation to simulate larger batch size
    # and using the GradScaler if data type is float16
    for micro_step in range(gradient_accumulation_steps):

        if ddp:
            # in DDP training we only need to sync gradients at the last micro step.
            # the official way to do this is with model.no_sync() context manager, but
            # I really dislike that this bloats the code and forces us to repeat code
            # looking at the source of that context manager, it just toggles this variable
            model.require_backward_grad_sync = (micro_step == gradient_accumulation_steps - 1)

        with ctx:
            logits, loss = model(X, Y)
            loss = loss / gradient_accumulation_steps # scale the loss to account for gradient accumulation

        # immediately async prefetch next batch while model is doing the forward pass on the GPU
        X, Y = get_batch('train')
        
        # backward pass, with gradient scaling if training in fp16
        scaler.scale(loss).backward()

    # clip the gradient
    if grad_clip != 0.0:
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)

    # step the optimizer and scaler if training in fp16
    scaler.step(optimizer)
    scaler.update()

    # flush the gradients as soon as we can, no need for this memory anymore
    optimizer.zero_grad(set_to_none=True)

    # timing and logging
    t1 = time.time()
    dt = t1 - t0
    t0 = t1
    if iter_num % log_interval == 0 and master_process:
        # get loss as float. note: this is a CPU-GPU sync point
        # scale up to undo the division above, approximating the true total loss (exact would have been a sum)
        lossf = loss.item() * gradient_accumulation_steps
        if local_iter_num >= 5: # let the training loop settle a bit
            mfu = raw_model.estimate_mfu(batch_size * gradient_accumulation_steps, dt)
            running_mfu = mfu if running_mfu == -1.0 else 0.9*running_mfu + 0.1*mfu
        print(f"iter {iter_num}: loss {lossf:.4f}, time {dt*1000:.2f}ms, mfu {running_mfu*100:.2f}%")
        
    iter_num += 1
    local_iter_num += 1

    # termination conditions
    if iter_num > max_iters:
        break

# CPU times: user 3min 24s, sys: 2.25 s, total: 3min 26s
# Wall time: 3min 31s

step 0: train loss 4.2883, val loss 4.2812


[2023-07-05 20:17:19,674] torch._inductor.utils: [WARNING] using triton random, expect difference from eager
[2023-07-05 20:17:19,898] torch._inductor.utils: [WARNING] using triton random, expect difference from eager
[2023-07-05 20:17:20,316] torch._inductor.utils: [WARNING] using triton random, expect difference from eager
[2023-07-05 20:17:20,531] torch._inductor.utils: [WARNING] using triton random, expect difference from eager
[2023-07-05 20:17:20,819] torch._inductor.utils: [WARNING] using triton random, expect difference from eager
[2023-07-05 20:17:21,037] torch._inductor.utils: [WARNING] using triton random, expect difference from eager
[2023-07-05 20:17:21,325] torch._inductor.utils: [WARNING] using triton random, expect difference from eager
[2023-07-05 20:17:21,542] torch._inductor.utils: [WARNING] using triton random, expect difference from eager
[2023-07-05 20:17:21,841] torch._inductor.utils: [WARNING] using triton random, expect difference from eager
[2023-07-05 20:17:2

iter 0: loss 4.2639, time 10385.03ms, mfu -100.00%
iter 10: loss 3.2853, time 28.87ms, mfu 3.23%
iter 20: loss 2.8315, time 29.10ms, mfu 3.22%
iter 30: loss 2.6489, time 28.85ms, mfu 3.22%
iter 40: loss 2.6118, time 28.81ms, mfu 3.23%
iter 50: loss 2.5976, time 28.79ms, mfu 3.23%
iter 60: loss 2.5761, time 28.87ms, mfu 3.23%
iter 70: loss 2.5549, time 28.83ms, mfu 3.23%
iter 80: loss 2.5225, time 28.81ms, mfu 3.23%
iter 90: loss 2.4923, time 28.86ms, mfu 3.23%
iter 100: loss 2.4482, time 28.83ms, mfu 3.23%
iter 110: loss 2.5309, time 28.76ms, mfu 3.23%
iter 120: loss 2.5103, time 28.80ms, mfu 3.23%
iter 130: loss 2.4700, time 28.79ms, mfu 3.23%
iter 140: loss 2.4821, time 28.69ms, mfu 3.23%
iter 150: loss 2.4872, time 28.71ms, mfu 3.23%
iter 160: loss 2.4667, time 29.00ms, mfu 3.23%
iter 170: loss 2.5109, time 28.80ms, mfu 3.23%
iter 180: loss 2.4728, time 28.77ms, mfu 3.23%
iter 190: loss 2.4521, time 28.88ms, mfu 3.23%
iter 200: loss 2.4686, time 28.86ms, mfu 3.23%
iter 210: loss 2.4

#### Sample.py

In [ ]:
endTime = time.time()
elapsedTime = time.strftime("%H:%M:%S", time.gmtime(endTime - startTime))
print(todaysDate.strftime('# Run Date: %A, %B %d, %Y'))
print(f"# Run Time: {elapsedTime}")